## Import

In [1]:
import pickle
import numpy as np 
import pandas as pd
import os 

from utilities_DL import get_DataSet_and_invalid_dates,get_MultiModel_loss_args_emb_opts,load_init_trainer
from DL_class import MultiModelTrainer, Trainer,PI_object,DataSet
from plotting_bokeh import generate_bokeh

from config import get_parameters
from paths import folder_path,file_name,save_folder
import torch

import copy 
from calendar_class import get_time_slots_labels

Training and Hyper-parameter tuning with Ray is not possible


## Choose the saved model : 
Here the 'best one', which mean the one on quantile loss with the smallest valid_loss. 

`Valid loss are not comparable for differents: `
-  K_fold
- fold
- single_station 
- dataset 
- validation


In [2]:
save_model_folder = f"{save_folder}best_models/"
model_perf_path = f"{save_model_folder}model_args.pkl"
model_perf = pickle.load(open(model_perf_path,'rb'))
best_valid_loss = np.inf 
for trial in model_perf['model'].keys():

    valid_loss = model_perf['model'][trial]['performance']['valid_loss']
    loss_function  = model_perf['model'][trial]['args']['loss_function_type']
    if loss_function == 'quantile':
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            best_trial = trial

best_model = f"{save_model_folder}{best_trial}_best_model.pkl"

print(best_model)

save/best_models/96850_STGCN_F1f0_64_30_011_28_05_2024_best_model.pkl


## Update some parameters

In [3]:
config = model_perf['model'][best_trial]['args']
config['abs_path'] = f"{os.path.abspath(os.getcwd())}/"
config['device'] = 'cuda:0' if torch.cuda.is_available() else 'cpu'
config['K_fold'] = 1
args = get_parameters(config)


## Load Empty model 

In [4]:
Datasets,DataLoader_list,dic_class2rpz,nb_words_embedding,time_slots_labels,dic_rpz2class = load_init_trainer(folder_path,file_name,args)
(loss_function,Model_list,Optimizer_list,Scheduler_list,args_embedding) = get_MultiModel_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz,n_vertex = len(Datasets[0].columns))
multimodeltrainer = MultiModelTrainer(Datasets,Model_list,DataLoader_list,args,Optimizer_list,loss_function,Scheduler_list,args_embedding=args_embedding,dic_class2rpz=dic_class2rpz,show_figure=False)

coverage period: 2019-01-01 00:00:00 - 2020-01-01 00:00:00
Time-step per hour: 4.0


## Load Weight

In [5]:
saved_checkpoint = torch.load(best_model)
trainer = multimodeltrainer.Trainers[0]
trainer.model.load_state_dict(saved_checkpoint['state_dict'])

<All keys matched successfully>

# Calibration

In [6]:
# Ici on regarde la calibration classique, celle qui ne fait aucune distinction selon les 'classes' considérée.
Q = trainer.conformal_calibration(trainer.alpha,trainer.dataset,conformity_scores_type = 'max_residual',quantile_method = 'classic',print_info = True)
pi,pi_cqr = generate_bokeh(trainer,trainer.dataloader,trainer.dataset,Q,args,dic_class2rpz,station=0,show_figure = True,save_plot = False)

# Qui est totalement équivalent à :
#calibration_calendar_class = 0
#Q = trainer.conformal_calibration(trainer.alpha,trainer.dataset,conformity_scores_type = 'max_residual',quantile_method = 'compute_quantile_by_class',print_info = True,calibration_calendar_class = calibration_calendar_class)
#pi,pi_cqr = generate_bokeh(trainer,trainer.dataloader,trainer.dataset,Q,args,dic_class2rpz,station=0,show_figure = True,save_plot = False,calibration_calendar_class = calibration_calendar_class)

Loading BokehJS ...

In [7]:
calibration_calendar_class = 3
Q = trainer.conformal_calibration(trainer.alpha,trainer.dataset,conformity_scores_type = 'max_residual',quantile_method = 'compute_quantile_by_class',print_info = True,calibration_calendar_class = calibration_calendar_class)
pi,pi_cqr = generate_bokeh(trainer,trainer.dataloader,trainer.dataset,Q,args,dic_class2rpz,station=0,show_figure = True,save_plot = False,calibration_calendar_class = calibration_calendar_class)


Proportion of label with quantile order set to 1: 0.0%


Loading BokehJS ...

In [8]:
calibration_calendar_class = 1
Q = trainer.conformal_calibration(trainer.alpha,trainer.dataset,conformity_scores_type = 'max_residual',quantile_method = 'compute_quantile_by_class',print_info = True,calibration_calendar_class = calibration_calendar_class)
pi,pi_cqr = generate_bokeh(trainer,trainer.dataloader,trainer.dataset,Q,args,dic_class2rpz,station=0,show_figure = True,save_plot = False,calibration_calendar_class = calibration_calendar_class)


Proportion of label with quantile order set to 1: 0.0%


Loading BokehJS ...

In [9]:
calibration_calendar_class = 2
Q = trainer.conformal_calibration(trainer.alpha,trainer.dataset,conformity_scores_type = 'max_residual',quantile_method = 'compute_quantile_by_class',print_info = True,calibration_calendar_class = calibration_calendar_class)
pi,pi_cqr = generate_bokeh(trainer,trainer.dataloader,trainer.dataset,Q,args,dic_class2rpz,station=0,show_figure = True,save_plot = False,calibration_calendar_class = calibration_calendar_class)


Proportion of label with quantile order set to 1: 96.7%


Loading BokehJS ...

## Pour l'instant, rien de très convainquant.

### Comment est traité 'Q' dans PI_object:

In [12]:
# Comment est traité 'Q' dans PI_object : 
# PI_object(preds,Y_true,alpha,Q = None,T_labels = None)
Q_tensor = torch.zeros(preds.size(0),preds.size(1),1).to(preds)
for label in T_labels.unique():
    indices = torch.nonzero(T_labels == label).squeeze()
    try: 
        Q_tensor[indices,:,0] = Q[label.item()]['Q'][0,:,0]
    except:
        print(f"No Conformal Calibration value found for {label.item()}. Will be set to 100") 
        Q_tensor[indices,:,0] = 100

torch.Size([40, 1])

In [14]:
# T_labels contient les CalendarClass. Ici de 0 à 167 inclus.
# Q est un dictionnaire. Chaque clé du dictionnaire est un label. A chaque Q[label] est associée la correction (commune pour lower et upper band).
# Exemple :  Q[label]['Q'] = [1,40,1]   (car 40 stations)
# Chaque correction est donc indépendante de la stations choisie.

tensor([108., 109., 109.,  ...,  47.,  47.,  47.])

### Calibration dans 'DL_class.Trainer':

In [ ]:
calendar_class = torch.cat([t_b for [_,_,t_b] in data])
    dic_label2Q = {}


    # Compute quantile for each calendar class : 
    nb_label_with_quantile_1 = 0
    for label in calendar_class.unique():
        indices = torch.nonzero(calendar_class == label,as_tuple = True)[0]
        quantile_order = torch.Tensor([np.ceil((1 - alpha)*(indices.size(0)+1))/indices.size(0)]).to(self.args.device)  # Quantile for each class, so the quantile order is different as each class has a different length
        quantile_order = min(torch.Tensor([1]).to(self.args.device),quantile_order)
        if quantile_order == 1: 
            nb_label_with_quantile_1 +=1
            #print(f"label {label} has only {indices.size(0)} elements in his class. We then use quantile order = 1")
        conformity_scores_i = self.conformity_scores[indices]
        scores_counts = conformity_scores_i.size(0)
        Q_i = get_higher_quantile(conformity_scores_i,quantile_order,device = self.args.device)
        #Q_i = torch.quantile(conformity_scores_i, quantile_order, dim = 0)#interpolation = 'higher'
        dic_label2Q[label.item()]= {'Q': Q_i,'count':scores_counts}

    str_info = str_info+ f"\nProportion of label with quantile order set to 1: {'{:.1%}'.format(nb_label_with_quantile_1/len(calendar_class.unique()))}"
    output = dic_label2Q

## Fonction à construire: 
- Il faut pouvoir repasser d'un calendar class (groupé par heure), à la représentation initiale (w,h,m).
- ET alors, on pourra refaire un nouveau groupement, une nouvelle labelisation.


# Sinon plutôt :
- Dès l'import, on génère toute les calendar_class possible. Elles sont dans un dictionnaire T_labels.
- On génère aussi un dictionnaire de dictionnaire dic_class2rpz/ dic_rpz2class
- Pour l'entrainement, on choisi T_labels[args.calendar_class]
- Pour la calibration, on a le choix de switcher à travers les labels que l'on veut 

In [175]:
from torch.utils.data import DataLoader,TensorDataset,ConcatDataset
import torch
dataloader = {}
feature_vector = torch.randn(12,3,4)
target = torch.randn(12,1,1)
L_time_slot = {0:torch.randn(12,2,2),
1:torch.randn(12,2,2),
2:torch.randn(12,2,2),
}


# convert L_time_slot en tuple :
time_slot_tensors =  list(L_time_slot.values())


nextiter = next(iter(dataloader['test']))

X,Y,*T = nextiter

X.size(),Y.size(),T[0].size(),T[1].size(),T[2].size()

In [ ]:
data = [[x_b,y_b,*t_b] for  x_b,y_b,*t_b in self.dataloader['cal']]

In [52]:
import pickle
import numpy as np 
import pandas as pd
import os 

from utilities_DL import get_MultiModel_loss_args_emb_opts,load_init_trainer
from DL_class import MultiModelTrainer

from config import get_parameters
from paths import folder_path,file_name,save_folder
import torch


def split_tensor(X,split_prop):
    return(X[:int(len(X)*split_prop)],X[int(len(X)*split_prop):])

def split_calibration_dataset(trainer,split_prop) :
    data = [[x_b,y_b,t_b[trainer.args.calendar_class]] for  x_b,y_b,*t_b in trainer.dataloader['cal']]

    X_cal = torch.cat([x_b for [x_b,_,_] in data]).to(trainer.args.device)
    Y_cal= torch.cat([y_b for [_,y_b,_] in data]).to(trainer.args.device)
    T_pred = torch.cat([t_pred for [_,_,t_pred] in data]).to(trainer.args.device)

    X_cal_train,X_cal_valid = split_tensor(X_cal,split_prop)
    Y_cal_train,Y_cal_valid = split_tensor(Y_cal,split_prop)
    T_pred_train,T_pred_valid = split_tensor(T_pred,split_prop)

    return(X_cal_train,X_cal_valid,Y_cal_train,Y_cal_valid,T_pred_train,T_pred_valid)

def prediction_on_specific_dataset(trainer,X,T):
    return(trainer.model(X,T))

def get_predictions(trainer,split_prop):
    X_cal_train,X_cal_valid,Y_cal_train,Y_cal_valid,T_pred_train,T_pred_valid = split_calibration_dataset(trainer,split_prop)
    
    Y_cal_train_pred = prediction_on_specific_dataset(trainer,X_cal_train,T_pred_train)
    Y_cal_valid_pred = prediction_on_specific_dataset(trainer,X_cal_valid,T_pred_valid)

    return(Y_cal_train_pred,Y_cal_train,Y_cal_valid_pred,Y_cal_valid)

def compute_error(Y_true,Y_pred_lower,Y_pred_upper):
    # 2 . ==== Compute error :     
    #   error_low = target - lower_band  / error_high  = upper_band - target
    #   err_dis = torch.cat([error_low,error_high])   # Concat errors on the dim 0 : [B,S,N],[B,S,N] -> [2*B,S,N]    , with S =nb step ahead of the prediction
    # ==== ....
    error_low = Y_true.cpu().detach() - Y_pred_lower.cpu().detach()
    error_high = Y_pred_upper.cpu().detach() - Y_true.cpu().detach()
    err =  torch.cat([error_low,error_high])

    return err 

def compute_quantile_of_residual_err_s_n(err,q,n,s):
    quantile_s_n = torch.quantile(err[:,n,s],q)
    return(quantile_s_n)

def compute_quantile_table_for_each_q(err,S,N,nb_quantiles = 99):
    quantile_table = torch.stack([torch.stack([torch.stack([compute_quantile_of_residual_err_s_n(err,q/(nb_quantiles+1),n,s) for s in range(S)]) for n in range(N)]) for q in range(nb_quantiles+1)])
    return(quantile_table)

def get_calibrated_metrics(Y_pred_lower,Y_pred_upper,quantile_table,nb_quantiles):
    calibrated_lower_band = Y_pred_lower.repeat(nb_quantiles+1,1,1,1).permute(1,0,2,3) - quantile_table   # calibrated band 
    calibrated_upper_band = Y_pred_upper.repeat(nb_quantiles+1,1,1,1).permute(1,0,2,3) - quantile_table
    
    # 4 . ==== PICP and MPIW Table:
    # For each quantile (percentile here, error_quantile = 99):
    #    Compute lower_band:  lower_prediction - quantile table for the quantile q    -> Shape [B,S,N]
    #    Compute upper_band:  upper_prediction + quantile table for the quantile q   -> Shape [B,S,N]  (on s'assure que lower_band > 0)
    #    Compute PICP (coverage_list) et MPIW (interval_list) mean through first dimension.  shape [S,N]
    # Compute it for each quantile  -> coverage_list and interval_list Shape [error_quantile,S,N]
    # ==== ....
    return(calibrated_PICP_table,calibrated_MPIW_table)

In [2]:
# ===== Load Dataset, Trainer, dataloader ...
save_model_folder = f"{save_folder}best_models/"
model_perf_path = f"{save_model_folder}model_args.pkl"
model_perf = pickle.load(open(model_perf_path,'rb'))
best_valid_loss = np.inf 
for trial in model_perf['model'].keys():

    valid_loss = model_perf['model'][trial]['performance']['valid_loss']
    loss_function  = model_perf['model'][trial]['args']['loss_function_type']
    if loss_function == 'quantile':
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            best_trial = trial

best_model = f"{save_model_folder}{best_trial}_best_model.pkl"

print(best_model)

config = model_perf['model'][best_trial]['args']
config['abs_path'] = f"{os.path.abspath(os.getcwd())}/"
config['device'] = 'cuda:0' if torch.cuda.is_available() else 'cpu'
config['K_fold'] = 1
args = get_parameters(config)

Datasets,DataLoader_list,dic_class2rpz,nb_words_embedding,time_slots_labels,dic_rpz2class = load_init_trainer(folder_path,file_name,args)
(loss_function,Model_list,Optimizer_list,Scheduler_list,args_embedding) = get_MultiModel_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz,n_vertex = len(Datasets[0].columns))
multimodeltrainer = MultiModelTrainer(Datasets,Model_list,DataLoader_list,args,Optimizer_list,loss_function,Scheduler_list,args_embedding=args_embedding,dic_class2rpz=dic_class2rpz,show_figure=False)
saved_checkpoint = torch.load(best_model)
trainer = multimodeltrainer.Trainers[0]
trainer.model.load_state_dict(saved_checkpoint['state_dict'])
# ===== ....

save/best_models/96850_STGCN_F1f0_64_30_011_28_05_2024_best_model.pkl
coverage period: 2019-01-01 00:00:00 - 2020-01-01 00:00:00
Time-step per hour: 4.0


<All keys matched successfully>

In [53]:
# ==== QuanTraffic Calibration : 
# Load Y_pred_calibration1 and Y_pred_calibration2
split_prop = 0.5
S = 1
N = 40
nb_quantiles = 99

Y_cal_train_pred,Y_cal_train,Y_cal_valid_pred,Y_cal_valid = get_predictions(trainer,split_prop)    # Get prediction from Calibration DataSet 
Y_cal_train_pred_lower,Y_cal_train_pred_upper = Y_cal_train_pred[...,0].unsqueeze(-1),Y_cal_train_pred[...,1].unsqueeze(-1)    # Get Upper and Lower band from prediction
Y_cal_valid_pred_lower,Y_cal_valid_pred_upper = Y_cal_valid_pred[...,0].unsqueeze(-1),Y_cal_valid_pred[...,1].unsqueeze(-1)

err = compute_error(Y_cal_train,Y_cal_train_pred_lower,Y_cal_train_pred_upper)  # Get Prediction  [2*B, N, 1] 
quantile_table = compute_quantile_table_for_each_q(err,S,N,nb_quantiles)

In [54]:
quantile_table.size(), Y_cal_train_pred_lower.size()

(torch.Size([100, 40, 1]), torch.Size([4524, 40, 1]))

In [59]:
().size()

torch.Size([4524, 100, 40, 1])

# Quantraffic

In [23]:
import torch

# Input Example : 
B,S,N = 16,6,4
B2 = 20
YS = torch.randn(B,S,N)
YS_pred_0 = torch.randn(B,S,N)
YS_pred_1 = torch.randn(B,S,N)

YS_val = torch.randn(B2,S,N)
YS_0_val = torch.randn(B2,S,N)
YS_1_val = torch.randn(B2,S,N)


errow_low = torch.randn(B,S,N).abs()
error_high = torch.randn(B,S,N).abs()
err_dis = torch.cat([errow_low,error_high])

error_quantile = 99  # 100 percentile 
lambda_list = np.arange(0,41)/41  # [0,..., 1]
exp_q = 0.9 # Expected quantile, i.e alpha = 0.1
# ...


corr_err_list = []
for q in range(0, error_quantile+1):
    q_n = []
    for n in range(N):
        q_t = []
        for t in range(S):
            corr_err = torch.quantile(err_dis[:, t, n], q/error_quantile)  # Quantile (nombre)
            q_t.append(corr_err)
        q_t = torch.stack(q_t)     # Quantile of residual for each 't' for a specific node i and percentile q
        q_n.append(q_t)
    q_n = torch.stack(q_n).T  # Quantile of residual for each 't' and each 'i' for a specific percentile q
    corr_err_list.append(q_n)  # Quantile for of residual for each 't' and each 'i' for each percentile 'q'
corr_err_list = torch.stack(corr_err_list)


print('Corr_err_list shape: ',corr_err_list.size()) # [100,L,N]


coverage_list = []
interval_list = []
for m in range(0, error_quantile+1):
    y_u_pred = YS_pred_0 + corr_err_list[m]
    y_l_pred = YS_pred_1 - corr_err_list[m]
    y_l_pred = y_l_pred * (y_l_pred >=0)
    mask = YS > 0
    # PICP
    coverage = torch.logical_and(y_u_pred >= YS, y_l_pred <= YS)
    coverage_a = torch.mean(coverage.float(),axis=0)
    coverage_list.append(coverage_a)
    # MPIW
    interval_a = torch.mean(torch.abs(y_u_pred - y_l_pred),axis=0)       
    interval_list.append(interval_a)
# torch.Size([sampe, seq, node])
coverage_list = torch.stack(coverage_list)
interval_list = torch.stack(interval_list)


print('PICP and MPIW table shape: ',coverage_list.size(),interval_list.size())

Corr_err_list shape:  torch.Size([100, 6, 4])
PICP and MPIW table shape:  torch.Size([100, 6, 4]) torch.Size([100, 6, 4])


In [24]:
def get_normalized_metrics(interval_list):
    interval_nor = []
    for n in range(N):
        interval_n = []
        for t in range(S):
            interval_ = (interval_list[:,t, n] - torch.min(interval_list[:,t, n])) /\
                        (torch.max(interval_list[:,t, n]) - torch.min(interval_list[:,t, n]))
            interval_n.append(interval_)
        interval_n = torch.stack(interval_n)
        interval_nor.append(interval_n)
    interval_nor = torch.stack(interval_nor).T

    print(interval_nor.size())
    return(interval_nor)

interval_nor = get_normalized_metrics(interval_list)
coverage_nor = get_normalized_metrics(coverage_list)


torch.Size([100, 6, 4])
torch.Size([100, 6, 4])


In [25]:
cor_err = []
lambda_list = torch.Tensor(lambda_list)
for i in lambda_list:
    loss = - i * coverage_nor + (1 - i) * interval_nor   # ICI PAS COMPRIS LA PONDERATION
    index = torch.argmin(loss,axis=0)
    err_t = []
    for t in range(S):
        err_n = []
        for n in range(N):
            corr_err = corr_err_list[index[t,n], t, n]
            err_n.append(corr_err)
        err_n = torch.stack(err_n)
        err_t.append(err_n)
    err_i = torch.stack(err_t)
    cor_err.append(err_i)
cor_err = torch.stack(cor_err)

print('cor_err: ',cor_err.size())

cor_err:  torch.Size([41, 6, 4])


In [43]:
# 选择期望的cor_err
independent_coverage_l =[]
# Pour chaque lambda, on prend la la valeur pour tout S et tout N sur la table de calibration, et on fait la calibration sur la sortie 
for i in range(len(lambda_list)): 
    y_u_pred = YS_0_val + cor_err[i]   #calibration
    y_l_pred = YS_1_val - cor_err[i]     #calibration

    y_l_pred = y_l_pred * (y_l_pred>0)   
    mask = YS_val>0     # Inutile car True partout ?????
    independent_coverage = torch.logical_and(torch.logical_and(y_u_pred >= YS_val, 
    y_l_pred <= YS_val), mask)   #PICP  ->  independent_coverage, Boolean Tensor, shape [B2,S,N]
    m_coverage = torch.sum(independent_coverage)/torch.sum(mask)  # Moyenne PICP sur la longueur de la Time-Serie, sur S et sur N -> Shape [1]
    independent_coverage_l.append(m_coverage)
m_coverage = torch.stack(independent_coverage_l)  # Vector [lambda_list], Contenant les PICP moyenne de la validation calibrée, pour chaque lambda
index = torch.argmin(torch.abs(m_coverage - exp_q))   # exp-q = 0.9 (i.e alpha  = 0.1)    -> On prend l'index (i.e le lambda), tel que la coverage est la plus proche de 0.9. 

print('m_coverage: ',m_coverage.size())
print('index: ',index.shape)

m_coverage:  torch.Size([41])
index:  torch.Size([])


In [ ]:
# Load Data
# Transform/Normalize Data
# Training

# Load Calibration dataset CalXS
# Calibration
err = calModel(MODELNAME, 'cal', calXS, calYS, scaler, lambda_list, q, n_grid)

# Load Test Dataset testXS
# Test model :
testunModel(MODELNAME, 'test', testXS, testYS, err, scaler)

In [ ]:

def calModel(name, mode, XS, YS,  scaler, lambda_list, exp_q, n_grid):
    '''
    this version is to compute our methods
    '''

    # 1 . ==== 'Preprocessing' 
    #   Data .to(device), DataLoader(XS,YS)
    #   Load trained model
    #   Prediction : YS_pred (quantile_l and quantile_u) on calibration dataset 
    #   Inverse_transform Prediction ( ??? )
 
    # split YS_pred in 2 part: upper and lower estimation (YS_pred_O and YS_pred1)
    #   split  YS and YS_pred in 2 part: 'train' and 'validation'  (YS_train,YS_val for the targets ones, and YS_0_train,YS_1_train and YS_0_val,YS_1_val for the predicted ones )
    #   Traffic state (demand, speed, flow) are always >= 0, then mask 'YS_1_train' which is the lower band, by setting all negative values by 0.

    #   error_quantile = 'n_grid'
    # ==== ....
    
    # 2 . ==== Compute error :     
    #   error_low = target - lower_band  / error_high  = upper_band - target
    #   err_dis = torch.cat([errow_low,error_high])   # Concat errors on the dim 0 : [B,S,N],[B,S,N] -> [2*B,S,N]    , with S =nb step ahead of the prediction
    # ==== ....

    # 3 . ==== Compute Quantile Table : 
    # Thanks to all residual error, compute a quantile for each percentile, each node id, each 't' 
    # corr_err_list = Quantile_table, shape : [error_quantile, S, N]
    # ==== ....

    # 4 . ==== PICP and MPIW Table:
    # For each quantile (percentile here, error_quantile = 99):
    #    Compute lower_band:  lower_prediction - quantile table for the quantile q    -> Shape [B,S,N]
    #    Compute upper_band:  upper_prediction + quantile table for the quantile q   -> Shape [B,S,N]  (on s'assure que lower_band > 0)
    #    Compute PICP (coverage_list) et MPIW (interval_list) mean through first dimension.  shape [S,N]
    # Compute it for each quantile  -> coverage_list and interval_list Shape [error_quantile,S,N]
    # ==== ....


    # 5 . ==== Compute PICP and MPIW MinMax_Normalization : (???) 
    #   Compute Normalized PICP  interval_nor (resp MPIW  - coverage_nor) through dim 0, For each node, each time-step ahead, 
    # ==== ....

    # 6 . ==== For each Lambda coeff, choose the best Quantile :
    #  For each lambda : 
    #    Loss = (1-i)MPIW_n  - i PICP_n    -> shape [error_quantile,S,N]
    #    find index table of min_loss for each couple (s,n) 
    #    for each s, each n : 
    #        cor_err[lambda,s,n] = Quantile_table[index[s,n],s,n]
    #
    # Return a Calibration table 'cor_err' of shape [lambda_list,S,N] 
    # ==== ....

    # 7 . ==== For each Lambda coeff, calibration on the Validation Prediction 
    #  For each lambda_i: 
    #    Compute lower band : lower_validation_predcition - calibration table[lambda_i]
    #    Compute upper band : upper_validation_predcition + calibration table[lambda_i]
    #    Compute coverage (independent_coverage), a Boolean Tensor, shape [B2,S,N]. True if real value within PI
    #    Compute PICP : mean of 'True coverage', shape [1]
    #
    # do for each lambda_i, then produce a PICP vector [lambda list]
    # Return the best lambda_i(the closest from expected quantile 0.9)
    # ==== .... 


    # 6 and 7 are the optimisation part to select the best_lambda
    # Then, for each 's' and each 'n', we have the associated best calibration cor_err[best_lambda,s,n]

    # These are the final calibration scores and can be used on test_dataset


def testunModel(name, mode, XS, YS, err, scaler):
    print('model uncertainty test')
    print('timestep_in, timestep_out', TIMESTEP_IN, TIMESTEP_OUT)
    XS_torch, YS_torch = torch.Tensor(XS).to(device), torch.Tensor(YS).to(device)
    test_data = torch.utils.data.TensorDataset(XS_torch, YS_torch)
    test_iter = torch.utils.data.DataLoader(test_data)
    model = getModel(name)
    model.load_state_dict(torch.load(PATH + '/' + name + '.pt'))

    YS_pred = predictModel(model, test_iter)
    YS_pred_0 = YS_pred[...,0]
    YS_pred_1 = YS_pred[...,1]

    YS, YS_pred_0, YS_pred_1 = np.squeeze(YS), np.squeeze(YS_pred_0), \
                               np.squeeze(YS_pred_1)
    YS, YS_pred_0, YS_pred_1 = inverse_transform(YS, scaler['mean'], scaler['std']), \
                               inverse_transform(YS_pred_0, scaler['mean'], scaler['std']), \
                              inverse_transform(YS_pred_1, scaler['mean'], scaler['std'])

    y_u_pred = YS_pred_0 + err[0]
    y_l_pred = YS_pred_1 - err[1]

    y_l_pred = y_l_pred * (y_l_pred > 0)
    mask = YS > 0
    independent_coverage = np.logical_and(np.logical_and(y_u_pred >= YS, y_l_pred <= YS), YS > 0)

    # compute the coverage and interval width
    results = {}
    results["Upper limit"] = np.array(y_l_pred)
    results["Lower limit"] = np.array(y_u_pred)
    results["Confidence interval widths"] = np.abs(y_u_pred - y_l_pred) * mask
    results["Mean confidence interval widths"] = np.sum(results["Confidence interval widths"]) / \
                                                 np.sum(mask)
    results["Independent coverage indicators"] = independent_coverage
    results["Mean independent coverage"] = np.sum(independent_coverage.astype(float)) / np.sum(mask)
    results["Calbration error"] = np.mean(err)

    with open(PATH + '/' + name + '_prediction_scores.txt', 'a') as f:
        f.write("calibration error,  %.4f\n "
                % results["Calbration error"])
        f.write("Mean independent coverage, Mean confidence interval widths, %.4f, %.4f\n "
                % (results["Mean independent coverage"], results["Mean confidence interval widths"]))

    print('*' * 40)
    print("Calibration error, %.4f\n" % np.mean(err[0] + err[1]))
    print("Mean independent coverage, Mean confidence interval widths, %.4f, %.4f\n "
          % (results["Mean independent coverage"], results["Mean confidence interval widths"]))